In [2]:
%pip install pandas
import pandas as pd
df = pd.read_csv('movies_dataset.csv')
data = df.to_dict('records')
df

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


,Sr No.,Movie Name,genre,Budget,Revenue,Profit,Release Year
0,1,Jurassic World,Action,150000000,1513528810,1363528810,2015
1,2,Mad Max: Fury Road,Action,150000000,378436354,228436354,2015
2,3,Insurgent,Thriller,110000000,295238201,185238201,2015
3,4,Star Wars: The Force Awakens,Action,200000000,2068178225,1868178225,2015
4,5,Furious 7,Action,190000000,1506249360,1316249360,2015
...,...,...,...,...,...,...,...
1241,1242,Live and Let Die,Action,7000000,161777836,154777836,1973
1242,1243,Thunderball,Action,11000000,141195658,130195658,1965
1243,1244,On Her Majesty's Secret Service,Action,7000000,81974493,74974493,1969
1244,1245,Halloween,Thriller,300000,70000000,69700000,1978


In [3]:
%pip install qdrant-client sentence-transformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

C:\Users\IND Gamers\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [6]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="top_movies",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\IND Gamers\AppData\Local\Temp\ipykernel_17488\4229820425.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [7]:
# vectorize!
qdrant.upload_points(
    collection_name="top_movies",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc['Movie Name']).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [8]:
user_prompt = "Some good horror movies"

In [9]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_movies",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'Sr No.': 600, 'Movie Name': 'Fright Night', 'genre': 'Comedy', 'Budget': 17000000, 'Revenue': 24922237, 'Profit': 7922237, 'Release Year': 2011} score: 0.5667700471650312
{'Sr No.': 1218, 'Movie Name': 'The Rocky Horror Picture Show', 'genre': 'Comedy', 'Budget': 1200000, 'Revenue': 112892319, 'Profit': 111692319, 'Release Year': 1975} score: 0.530821020757575
{'Sr No.': 806, 'Movie Name': 'House of the Dead', 'genre': 'Action', 'Budget': 7000000, 'Revenue': 13818181, 'Profit': 6818181, 'Release Year': 2003} score: 0.5216741784642254


C:\Users\IND Gamers\AppData\Local\Temp\ipykernel_17488\3087370386.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [10]:
search_results = [hit.payload for hit in hits]

In [11]:
%pip install openai
from openai import OpenAI

client = OpenAI(
    base_url="http://127.0.0.1:8080/v1",  # <-- fixed
    api_key="sk-no-key"                   # <-- dummy key if not required
)

completion = client.chat.completions.create(
    model="Llama-3.2-3B-Instruct.Q6_K4_2024-09-24",  # your local model name
    messages=[
        {"role": "system", "content": "You are a helpful assistant that helps people find movies."},
        {"role": "user", "content": f"Find some good movies based on these recommendations: {search_results}"}
    ]
)

print(completion.choices[0].message.content)  # or .content if SDK is new


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Based on your recommendations, I've found some good movies that match the genres and release years. Here are a few suggestions:

**Comedy Movies:**

1. **The Addams Family (1991)** - A classic comedy film based on the popular TV series, with a similar tone to "Fright Night" and "The Rocky Horror Picture Show".
2. **Ghostbusters (1984)** - A supernatural comedy classic that shares a similar blend of humor and horror with "Fright Night".
3. **Young Frankenstein (1974)** - A Mel Brooks comedy that parodies the classic horror genre, similar to "The Rocky Horror Picture Show".
4. **Tucker & Dale vs Evil (2010)** - A horror-comedy that spoofs hillbilly horror movies, with a similar irreverent tone to "Fright Night".
5. **Shaun of the Dead (2004)** - A zombie romp that combines horror and comedy, with a similar irreverent tone to "The Rocky Horror Pic